In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import repaint_lib
from torchvision.utils import make_grid
from tqdm import tqdm

workdir = "./"
repaint_dir = os.path.join(workdir, "repaint")

gamma = 5
s = repaint_lib.get_s(gamma=gamma)

In [ ]:
for idx in os.listdir(repaint_dir):
    idx_dir = os.path.join(repaint_dir, idx)

    x0 = torch.load(os.path.join(idx_dir, "original.pt"))

    for _s in tqdm(s[::-1]):
        _s_name = "".join([str(u) for u in _s])
        _s_dir = os.path.join(idx_dir, _s_name)
        checkpoint_dir = os.path.join(_s_dir, "checkpoints")

        repainted_filename = os.path.join(_s_dir, "repainted.pt")
        if os.path.exists(repainted_filename):
            m = torch.load(os.path.join(_s_dir, "mask.pt"))
            repainted = torch.load(repainted_filename)

            x = m * x0

            data = torch.cat([x0, x, repainted], dim=0)
            data = torch.clamp(data, 0, 1)
            im = make_grid(data, nrow=9)

            _, ax = plt.subplots(figsize=(16, 9))
            ax.imshow(im.permute(1, 2, 0))
            ax.axis("off")
            ax.set_title(f"S = {_s}")
            plt.savefig(os.path.join(_s_dir, "results.pdf"), bbox_inches="tight")
            plt.savefig(os.path.join(_s_dir, "results.jpg"), bbox_inches="tight")
            plt.close()

            steps = os.listdir(checkpoint_dir)
            checkpoints = torch.stack(
                [torch.load(os.path.join(checkpoint_dir, s)) for s in steps], dim=1
            )

            data = torch.cat([x0, x, checkpoints[0]], dim=0)
            data = torch.clamp(data, 0, 1)
            im = make_grid(data, nrow=9)

            _, ax = plt.subplots(figsize=(16, 9))
            ax.imshow(im.permute(1, 2, 0))
            ax.axis("off")
            ax.set_title(f"S = {_s}")
            plt.savefig(os.path.join(_s_dir, "checkpoints.pdf"), bbox_inches="tight")
            plt.savefig(os.path.join(_s_dir, "checkpoints.jpg"), bbox_inches="tight")
            plt.close()